Imports

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import Secrets
import asyncio
import aiohttp
import threading
import time
import re

Fetching Data from VGChartz

In [ ]:
list_of_games = []
def search_for_sales(page_number):
    sales_url= "https://www.vgchartz.com/games/games.php?page="+str(page_number)+"&results=200&order=TotalSales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=0&showpalsales=0&showjapansales=0&showothersales=0&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=0&showcriticscore=0&showuserscore=0&showshipped=1"
    response_sales_data= requests.get(sales_url)
    # Parse the HTML content with Beautiful Soup
    soup = BeautifulSoup(response_sales_data.content, 'html.parser')
    games = soup.find_all('td')
    game_names = [element.find('a').get_text() for element in games if element.find('a') and element.get_text(strip=True)]
    total_shipped_elements = [element.get_text() for element in games if element.get_text().endswith('m')]
    game_index = 0
    for game_name in game_names:
        game_name = game_name.strip()
        if game_name != "Add new game":
            if game_index < len(total_shipped_elements):
                game_dict = {
                    "name": re.sub(r'\([^)]*\)', '', game_name.strip().lower()),
                    "total_shipped": float(total_shipped_elements[game_index].replace("m", ""))
                    }
                if game_dict["name"].strip().lower() != list_of_games:
                    list_of_games.append(game_dict)
                game_index+=1
threads = []
num_of_pages = int(63076/200)
for page_number in range(1,num_of_pages+1):
    game_thread = threading.Thread(target=search_for_sales,args=(page_number,))
    game_thread.start()
    threads.append(game_thread)
    # pause for web scraping limits to not give read limit errors
    time.sleep(2)
for thread in threads:
    thread.join()
dataframe = pd.DataFrame(list_of_games, columns=["name", "total_shipped"])

Fetching Data from IGDB

In [ ]:
auth_url = 'https://id.twitch.tv/oauth2/token?client_id=gq83s6s46z91nf6hf354n9eyqk0ev5&client_secret='+Secrets.Secret_variables.client_secret+'&grant_type=client_credentials'
game_criteria_url = 'https://api.igdb.com/v4/games'
response = requests.post(auth_url)
auth_token = response.json()
json_data = []
async def fetch_data(session, url, payload, headers):
    async with session.post(url, data=payload, headers=headers) as response:
        return await response.json()
    

async def main():
    global auth_token
    auth_url = 'https://id.twitch.tv/oauth2/token?client_id=gq83s6s46z91nf6hf354n9eyqk0ev5&client_secret='+Secrets.Secret_variables.client_secret+'&grant_type=client_credentials'
    game_criteria_url = 'https://api.igdb.com/v4/games'
    headers = {'Client-ID': Secrets.Secret_variables.client_id, 'Authorization': "Bearer "+auth_token["access_token"]}
    tasks = []

    async with aiohttp.ClientSession() as session:
        # Fetch auth token
        async with session.post(auth_url) as response:
            auth_token = (await response.json())["access_token"]
            headers['Authorization'] = f'Bearer {auth_token}'

        for game_name in dataframe["name"]:
            payload = 'search \"'+ game_name +'\"; fields name,alternative_names,age_ratings.rating,game_engines.name, platforms.name, game_modes.name, genres.name,themes.name; exclude alternative_names; where release_dates.y > 2000 & age_ratings.category = 1 & age_ratings.rating != null & game_engines.name != null & platforms.name != null & game_modes.name != null & genres.name != null & themes.name != null;'
            task = asyncio.ensure_future(fetch_data(session, game_criteria_url, payload, headers))
            tasks.append(task)
            # pause for web scraping limits to not give read limit errors
            await asyncio.sleep(0.25)

        responses = await asyncio.gather(*tasks)

        # Process responses here
        for response in responses:
            json_data.append(response)
await main()

Load data into Dict

In [ ]:
new_game_data = []
rating_mapping = {
    6: "RP",
    7: "EC",
    8: "E",
    9: "E10",
    10: "T",
    11: "M",
    12: "AO"
}

for game_entry in json_data:
    if len(game_entry) > 0:
        if "title" not in game_entry[0].keys():
            game_entry = game_entry[0]
            game_dict = {
                "name": game_entry["name"].strip().lower(),
                "genres": [genre["name"].strip().lower() for genre in game_entry.get("genres", [])],
                "themes": [theme["name"].strip().lower() for theme in game_entry.get("themes", [])],
                "game_engines": str(game_entry["game_engines"][0]["name"]).strip().lower() if "game_engines" in game_entry else "Unknown",
                "age_ratings": "".join([
                    rating_mapping[age_rating["rating"]].strip().lower()
                    for age_rating in game_entry.get("age_ratings", [])
                    if age_rating["rating"] in rating_mapping 
                ]),
            }
            new_game_data.append(game_dict)

Store into Dataframe and write to CSV

In [ ]:
old_dataframe = dataframe
new_dataframe = pd.DataFrame(new_game_data, columns=["name", "genres", "themes", "game_engines", "age_ratings"])
dataframe = pd.merge(old_dataframe,new_dataframe,on="name", how="left")
dataframe = dataframe.drop_duplicates(subset=["name"])
dataframe = dataframe.dropna()
print(dataframe)
dataframe.to_csv('videogame_data.csv',index = False)